In [1]:
import numpy as np

In [ ]:
class MM:
    def __init__(self, in_dim, out_dim):
        self.in_dim = in_dim
        self.out_dim = out_dim
        
        self.W = np.random.rand(self.out_dim, self.in_dim)
        self.b = np.random.rand(out_dim, 1)

        self.layer_type = "MM"

    def forward(self, z):
        return np.matmul(self.W, z) + self.b
    
    def backward(self, v):
        return np.matmul(self.W.T, v)
    
    def update_w_b(self, z, v, lr):
        dw = np.matmul(v, z.T)
        db = v

        self.W = self.W - (lr * dw)
        self.b = self.b - (lr * db)

In [ ]:
class ReLU:
    def __init__(self, in_dim):
        self.in_dim = in_dim
        self.layer_type = "ReLU"

    def forward(self, z):
        return np.maximum(z, 0)
    
    def backward(self, z, v):
        dz = np.zeros(z.shape)
        for i in range(z.shape[0]):
            if(z[i, 0] > 0):
                dz[i, 0] = v[i, 0]
        
        return dz
        

In [ ]:
def softmax(z):
    e_z = np.exp(z - np.max(z))  # for numerical stability
    return e_z / np.sum(e_z)

In [46]:
class CE_loss:
    def __init__(self, in_dim):
        self.in_dim = in_dim

    def forward(self, z, y):
        SM_z = softmax(z)

        J = 0
        for i in range(self.in_dim):
            J -= int(y[i, 0] == 1) * np.log(SM_z[i, 0])
        
        return J
    
    def backward(self, z, y):
        return softmax(z) - y

In [1]:
class NN:
    def __init__(self, layers, loss, lr):
        self.layers = layers
        self.loss = loss
        self.lr = lr

    def train(self, x, y):
        outputs = [x]
        for layer in self.layers:
            outputs.append(layer.forward(outputs[-1]))
        
        J = self.loss.forward(outputs[-1], y)

        backwards = [self.loss.backward(outputs[-1], y)]
        self.layers[-1].update_w_b(outputs[-2], backwards[-1], self.lr)

        for i in range(len(self.layers) - 2, -1, -1):
            layer = self.layers[i]
            
            if layer.layer_type == "MM":
                backwards.append(layer.backward(backwards[-1]))
                layer.update_w_b(outputs[i], backwards[-1], self.lr)
            
            if layer.layer_type == "ReLU":
                backwards.append(layer.backward(outputs[i - 1], backwards[-1]))

        return J